# Graphing Calculator (Python + Colab)

Notebook này tự chứa mã cần thiết để chạy độc lập trên Google Colab. Chạy lần lượt các cell.

In [ ]:
# Cài đặt phụ thuộc
!pip -q install sympy==1.13.2 numpy==1.26.4 matplotlib==3.8.4

## Tạo `graphing.py` trong runtime Colab
Nếu mở notebook đơn lẻ, cell này sẽ tạo module `graphing.py`.

In [ ]:
%%writefile graphing.py
from __future__ import annotations
from typing import Iterable, List, Optional, Tuple

import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

x, y = sp.symbols('x y')

def parse_functions(func_strs: Iterable[str]) -> List[sp.Expr]:
    exprs: List[sp.Expr] = []
    for s in func_strs:
        exprs.append(sp.sympify(s, convert_xor=True))
    return exprs

def plot_functions(
    func_strs: Iterable[str],
    x_range: Tuple[float, float] = (-10, 10),
    shade: Optional[Tuple[str, str]] = None,  # ('above'|'below', func_str)
    zoom: Optional[Tuple[float, float, float, float]] = None,  # (xmin,xmax,ymin,ymax)
    ax: Optional[plt.Axes] = None,
) -> plt.Axes:
    exprs = parse_functions(func_strs)
    f_lambdas = [sp.lambdify(x, e, 'numpy') for e in exprs]

    xmin, xmax = x_range
    xs = np.linspace(xmin, xmax, 1000)

    if ax is None:
        fig, ax = plt.subplots(figsize=(7, 5))

    for s, f in zip(func_strs, f_lambdas):
        ys = f(xs)
        ax.plot(xs, ys, label=s)

    if shade is not None:
        mode, shade_func_str = shade
        shade_expr = sp.sympify(shade_func_str, convert_xor=True)
        shade_lambda = sp.lambdify(x, shade_expr, 'numpy')
        ys_shade = shade_lambda(xs)
        if mode.lower() == 'above':
            ax.fill_between(xs, ys_shade, ax.get_ylim()[1], alpha=0.2, color='C0')
        elif mode.lower() == 'below':
            ax.fill_between(xs, ax.get_ylim()[0], ys_shade, alpha=0.2, color='C0')
        else:
            raise ValueError("shade mode must be 'above' or 'below'")

    ax.axhline(0, color='black', linewidth=0.8)
    ax.axvline(0, color='black', linewidth=0.8)
    ax.grid(True, alpha=0.3)
    ax.legend(loc='best')

    if zoom is not None:
        ax.set_xlim(zoom[0], zoom[1])
        ax.set_ylim(zoom[2], zoom[3])

    return ax

def table_xy(func_str: str, x_values: Iterable[float]):
    expr = sp.sympify(func_str, convert_xor=True)
    f = sp.lambdify(x, expr, 'numpy')
    table = []
    for xv in x_values:
        yv = float(f(xv))
        table.append((float(xv), yv))
    return table

def solve_and_plot_system(
    eq_strs: Iterable[str],
    x_range=(-10, 10),
    y_range=(-10, 10),
    ax: Optional[plt.Axes] = None,
):
    equations = []
    for s in eq_strs:
        if '=' in s:
            left, right = s.split('=', 1)
            equations.append(sp.sympify(left) - sp.sympify(right))
        else:
            equations.append(sp.sympify(s))

    sols = sp.solve(equations, (x, y), dict=True)
    pts = []
    for sol in sols:
        pts.append((float(sol[x]), float(sol[y])))

    if ax is None:
        fig, ax = plt.subplots(figsize=(7, 5))
    u = np.linspace(x_range[0], x_range[1], 400)
    v = np.linspace(y_range[0], y_range[1], 400)
    U, V = np.meshgrid(u, v)
    for eq in equations:
        f = sp.lambdify((x, y), eq, 'numpy')
        Z = f(U, V)
        ax.contour(U, V, Z, levels=[0], colors=['C1'], linewidths=1.2)

    if pts:
        Xp, Yp = zip(*pts)
        ax.scatter(Xp, Yp, color='red', zorder=5, label='solutions')

    ax.set_xlim(*x_range)
    ax.set_ylim(*y_range)
    ax.axhline(0, color='black', linewidth=0.8)
    ax.axvline(0, color='black', linewidth=0.8)
    ax.grid(True, alpha=0.3)
    ax.legend(loc='best')
    return pts, ax

def solve_quadratic(a: float, b: float, c: float):
    A, B, C = map(sp.sympify, (a, b, c))
    roots = sp.solve(sp.Eq(A*x**2 + B*x + C, 0), x)
    return roots


## Ví dụ sử dụng

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from graphing import (parse_functions, plot_functions, table_xy, solve_and_plot_system, solve_quadratic)

# 1) Vẽ nhiều hàm và tô vùng
ax = plot_functions(["x**2", "2*x+1"], x_range=(-5, 5), shade=('below', '2*x+1'))
plt.show()

# 2) Bảng (x,y)
tbl = table_xy('sin(x)', np.linspace(0, np.pi, 5))
print('Table sin(x):', tbl)

# 3) Giải & vẽ hệ phương trình
pts, ax = solve_and_plot_system(["y = x + 1", "y = -x + 3"], x_range=(-1, 4), y_range=(-1, 4))
print('Solutions:', pts)
plt.show()

# 4) Zoom
ax = plot_functions(["cos(x)"], x_range=(-10, 10), zoom=(-3, 3, -1, 1))
plt.show()

# 5) Giải bậc hai
print('Quadratic roots of x^2-3x+2:', solve_quadratic(1, -3, 2))


## Kiểm thử nhanh

In [ ]:
import sympy as sp
from graphing import parse_functions, table_xy, solve_quadratic

# parse_functions
exprs = parse_functions(['x**2', 'sin(x)'])
assert len(exprs) == 2 and all(isinstance(e, sp.Expr) for e in exprs)

# table_xy
tbl = table_xy('x**2', [0, 1, 2])
assert tbl == [(0.0, 0.0), (1.0, 1.0), (2.0, 4.0)]

# solve_quadratic
roots = solve_quadratic(1, -3, 2)
assert set(map(sp.simplify, roots)) == set(map(sp.simplify, [1, 2]))

print('Tất cả kiểm thử đã PASSED!')
